In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
import os
import cv2


/var/folders/83/_p14qyt53xl7cpdfv7wby79r0000gn/T/ipykernel_91616/1233727652.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-14 20:17:52.816568: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
from tensorflow.keras.models import load_model


In [14]:
import cv2
import os

import numpy as np
from collections import Counter

In [15]:
import pytesseract
from PIL import Image

# Install pytesseract using pip if not already installed
# pip install pytesseract

# Install Tesseract OCR engine: https://github.com/tesseract-ocr/tesseract
# Make sure to add the Tesseract installation directory to your system PATH

def extract_text_from_image(image):
    # Open the image using PIL (Python Imaging Library)

    # Perform OCR using pytesseract
    text = pytesseract.image_to_string(image)

    return text

In [16]:

image_folder = '/Users/ritikgupta/Downloads/6_end_to_end/6_end_to_end/'
# image_folder ='/content/drive/MyDrive/Colab Notebooks/1_classification/Test/Non_Ids/'

In [17]:
def deskew_with_double_majority(original_image):
    # Load the image
    # original_image = cv2.imread(image_path)

    # Convert the image to grayscale

    gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion
    kernel = np.ones((25, 25), np.uint8)
    dilation = cv2.dilate(gray, kernel, iterations=2)
    erosion = cv2.erode(dilation, kernel, iterations=1)
    new= erosion*gray


    # Use Canny edge detection
    edges = cv2.Canny(erosion, 50, 150, apertureSize=3)

    # Apply Hough Line Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 100)

    if lines is None or len(lines) < 2:
        raise ValueError("No lines detected for deskewing")

    # Extract angles from detected lines
    angles = [line[0][1] for line in lines]

    # First majority vote: Determine lines
    line_votes = Counter(angles)
    common_lines = line_votes.most_common(2)
    majority_line = common_lines[0][0]

    # Second majority vote: Vote on a common angle among the most important lines
    important_lines = [line for line in lines if line[0][1] == majority_line]
    important_angles = [line[0][1] for line in important_lines]

    angle_votes = Counter(important_angles)
    common_angles = angle_votes.most_common(2)
    majority_angle = common_angles[0][0]

    # Rotate the image to deskew it
    (h, w) = original_image.shape[:2]
    center = (w // 2, h // 2)
    rotation_angle = np.degrees(majority_angle) - 90
    M = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)
    deskewed_image = cv2.warpAffine(original_image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    deskewed_image_gray = cv2.warpAffine(new, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)


    return(deskewed_image,deskewed_image_gray)

In [18]:
def segmentation(original_image):
    image_resized= tf.keras.preprocessing.image.img_to_array(original_image)/255.0
    image_with_batch = np.expand_dims(image_resized, axis=0)
    segmentation_mask = model1.predict(image_with_batch)
    cut_out = image_resized  * segmentation_mask
    cut_out=cut_out.reshape((512, 512, 3))
    cut_out = (cut_out * 255).astype(np.uint8)
    return cut_out

In [19]:
def classify_image(image):
    # Perform inference
    prediction = model.predict(np.expand_dims(image, axis=0))
    return prediction


In [20]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

def run_pipeline(original_image):
    # Step 2: Classify the image
    # cv2_imshow(original_image)
    prediction = classify_image(original_image)

    # Step 3: Post-process the prediction
    if prediction <= 0.4:
        print(prediction, "ID card detected.")
        # # Step 4: Load the segmentation model and predict


        cut_out = segmentation(original_image)
        print("image============================================================")
#         cv2.imshow('2',cut_out)
#         cv2.waitKey()

        deskew_image,deskew_image_new = deskew_with_double_majority(cut_out)
        print("deskew_image================================================== ")
#         cv2.imshow('3',deskew_image)
#         cv2.waitKey()

        print("deskew_image morphological filters=============================")
#         cv2.imshow('4',deskew_image_new)
#         cv2.waitKey()
        image_with_batch = np.expand_dims(deskew_image, axis=0)

        pred= model2.predict(image_with_batch)
        pred = np.squeeze(pred)
        pred = (pred*255).astype(np.uint8)
#         cv2.imshow('4',deskew_image_new)
#         cv2.waitKey()
        text=extract_text_from_image(cut_out)
        print(text,"nkn")

    else:
        print(prediction, "Not an ID card.")


In [21]:
model = load_model(r'/Users/ritikgupta/IPDA 23/All_trained_models/id_card_classifier2.h5',compile=False)
model1 = load_model('/Users/ritikgupta/IPDA 23/All_trained_models/id_card_seg1.h5')
model2 = load_model('/Users/ritikgupta/IPDA 23/All_trained_models/id_card_clean.h5')


In [22]:

for filename in os.listdir(image_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_folder, filename)
        original_image = cv2.imread(image_path)
        image = cv2.resize(original_image, (512, 512))
        # original_image = original_image / 255.0
        print("orignal image")
#         cv2.imshow('1',image)
#         cv2.waitKey()

        # print(image)
        run_pipeline(image)
     
        cv2.destroyAllWindows()


        break


orignal image
1/1 [==============================] - 0s 346ms/step
[[0.01543304]] ID card detected.
1/1 [==============================] - 1s 1s/step
image============================================================
deskew_image================================================== 
deskew_image morphological filters=============================
1/1 [==============================] - 2s 2s/step
 nkn
